In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

In [5]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

edu_data = pd.read_sql_query('select * from useducation', con=engine)

edu_data.info()
edu_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1492 non-null   object 
 1   STATE                         1492 non-null   object 
 2   YEAR                          1492 non-null   int64  
 3   ENROLL                        1229 non-null   float64
 4   TOTAL_REVENUE                 1280 non-null   float64
 5   FEDERAL_REVENUE               1280 non-null   float64
 6   STATE_REVENUE                 1280 non-null   float64
 7   LOCAL_REVENUE                 1280 non-null   float64
 8   TOTAL_EXPENDITURE             1280 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1280 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1280 non-null   float64
 11  OTHER_EXPENDITURE             1229 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1280 non-null   float64
 13  GRA

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,NaN,174053.0,8224.0,55460.0,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,NaN,37451.0,2371.0,10152.0,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,NaN,609114.0,2544.0,53497.0,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,NaN,145212.0,808.0,33511.0,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,NaN,2044688.0,59067.0,431763.0,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


In [15]:
floats= []

for col in edu_data:
  if edu_data[col].dtype == 'float64':
   floats.append(col)

for col in floats:
  edu_data.loc[:,col] = edu_data.loc[:,col].fillna(edu_data.loc[:,col].mean())

edu_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1492 non-null   object 
 1   STATE                         1492 non-null   object 
 2   YEAR                          1492 non-null   int64  
 3   ENROLL                        1492 non-null   float64
 4   TOTAL_REVENUE                 1492 non-null   float64
 5   FEDERAL_REVENUE               1492 non-null   float64
 6   STATE_REVENUE                 1492 non-null   float64
 7   LOCAL_REVENUE                 1492 non-null   float64
 8   TOTAL_EXPENDITURE             1492 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1492 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1492 non-null   float64
 11  OTHER_EXPENDITURE             1492 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1492 non-null   float64
 13  GRA

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,GRADES_PK_G,GRADES_KG_G,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,1.005984e+06,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,735036.0,500428.823489,174053.0,8224.0,55460.0,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,263.267799
1,1992_ALASKA,ALASKA,1992,1.005984e+06,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,350902.0,500428.823489,37451.0,2371.0,10152.0,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,211.790904,258.776900,207.088116,258.859712
2,1992_ARIZONA,ARIZONA,1992,1.005984e+06,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,1007732.0,500428.823489,609114.0,2544.0,53497.0,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,1.005984e+06,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,483488.0,500428.823489,145212.0,808.0,33511.0,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,1.005984e+06,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,8520926.0,500428.823489,2044688.0,59067.0,431763.0,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,265.519676


In [19]:
grade_4_data = edu_data['GRADES_4_G']*(edu_data['AVG_MATH_4_SCORE'] + edu_data['AVG_READING_4_SCORE'])
grade_8_data = edu_data['GRADES_8_G']*(edu_data['AVG_MATH_8_SCORE'] + edu_data['AVG_READING_8_SCORE'])
edu_data['total average score'] = (grade_4_data + grade_8_data)/(edu_data['GRADES_4_G'] + edu_data['GRADES_8_G'])

edu_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 26 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   PRIMARY_KEY                   1492 non-null   object 
 1   STATE                         1492 non-null   object 
 2   YEAR                          1492 non-null   int64  
 3   ENROLL                        1492 non-null   float64
 4   TOTAL_REVENUE                 1492 non-null   float64
 5   FEDERAL_REVENUE               1492 non-null   float64
 6   STATE_REVENUE                 1492 non-null   float64
 7   LOCAL_REVENUE                 1492 non-null   float64
 8   TOTAL_EXPENDITURE             1492 non-null   float64
 9   INSTRUCTION_EXPENDITURE       1492 non-null   float64
 10  SUPPORT_SERVICES_EXPENDITURE  1492 non-null   float64
 11  OTHER_EXPENDITURE             1492 non-null   float64
 12  CAPITAL_OUTLAY_EXPENDITURE    1492 non-null   float64
 13  GRA

In [24]:
expenditure = ['TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE']
edu_data[['TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE','total average score']].corr()

,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,total average score
TOTAL_EXPENDITURE,1.000000,0.992757,0.992496,0.945975,0.928443,0.210979
INSTRUCTION_EXPENDITURE,0.992757,1.000000,0.979327,0.914713,0.896026,0.213350
SUPPORT_SERVICES_EXPENDITURE,0.992496,0.979327,1.000000,0.947211,0.905785,0.228082
OTHER_EXPENDITURE,0.945975,0.914713,0.947211,1.000000,0.919115,0.162977
CAPITAL_OUTLAY_EXPENDITURE,0.928443,0.896026,0.905785,0.919115,1.000000,0.137394
total average score,0.210979,0.213350,0.228082,0.162977,0.137394,1.000000


In [31]:
X = edu_data[expenditure]
X = StandardScaler().fit_transform(X)
sklearn_pca = PCA(n_components=1)
edu_data['pca_1'] = sklearn_pca.fit_transform(X)
print(sklearn_pca.explained_variance_ratio_)


[0.95398481]


In [32]:
edu_data[['pca_1','TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE', 'CAPITAL_OUTLAY_EXPENDITURE','total average score']].corr()

,pca_1,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,total average score
pca_1,1.000000,0.995425,0.979863,0.988386,0.967836,0.951477,0.195592
TOTAL_EXPENDITURE,0.995425,1.000000,0.992757,0.992496,0.945975,0.928443,0.210979
INSTRUCTION_EXPENDITURE,0.979863,0.992757,1.000000,0.979327,0.914713,0.896026,0.213350
SUPPORT_SERVICES_EXPENDITURE,0.988386,0.992496,0.979327,1.000000,0.947211,0.905785,0.228082
OTHER_EXPENDITURE,0.967836,0.945975,0.914713,0.947211,1.000000,0.919115,0.162977
CAPITAL_OUTLAY_EXPENDITURE,0.951477,0.928443,0.896026,0.905785,0.919115,1.000000,0.137394
total average score,0.195592,0.210979,0.213350,0.228082,0.162977,0.137394,1.000000
